# Install/import packages

In [1]:
!pip install pymatgen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 21.9 MB/s eta 0:00:00


In [2]:
!pip install mp_api

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.5/874.5 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 17.1 MB/s eta 0:00:00
   ━

In [3]:
import pymatgen
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core.periodic_table import Element
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from mp_api.client import MPRester
from pymatgen.core import Structure
import numpy as np
import pandas as pd

# Creating .csv file with all info

In [13]:
API_KEY = "mhz7IO0VGT4oZ7S8qu1pHsPL4zOQw7kx"

In [17]:
def compute_descriptors(structure):
    """Compute the five key structural descriptors from the paper"""
    results = {}

    # Initialize neighbor analysis tool
    cnn = CrystalNN()

    # Get Li sites
    li_sites = [site for site in structure if site.species_string == "Li"]

    # 1. Li-Li Bond Count (LLB)
    li_bonds = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        li_neighbors = [n for n in neighbors if n["site"].species_string == "Li"]
        li_bonds += len(li_neighbors)
    results["LLB"] = li_bonds / len(li_sites) if li_sites else 0

    # 2. Sublattice Bond Ionicity (SBI)
    #anion_elements = {'As', 'B', 'Br', 'C', 'Cl', 'F', 'Ge', 'H', 'I', 'N', 'O', 'P', 'S', 'Sb', 'Se', 'Si', 'Te', 'Xe'}
    ionicity_sum = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        for neighbor in neighbors:
            #if neighbor["site"].species_string in anion_elements:
            delta_X = abs(Element("Li").X - Element(neighbor["site"].species_string).X)
            ionicity_sum += delta_X
    results["SBI"] = ionicity_sum / len(structure) if len(structure) > 0 else 0

    # 3. Anion Framework Coordination (AFC)
    anions = [site for site in structure] #if site.species_string in anion_elements]
    total_coord = sum(len(cnn.get_nn_info(structure, structure.index(anion))) for anion in anions)
    results["AFC"] = total_coord / len(anions) if anions else 0

    # 4. Li-Anion Separation Distance (LASD)
    anion_sites = [site for site in structure] #if site.species_string in anion_elements]
    if li_sites and anion_sites:
        distances = [li.distance(anion) for li in li_sites for anion in anion_sites]
        results["LASD"] = np.mean(distances)
    else:
        results["LASD"] = 0

    # 5. Li-Li Separation Distance (LLSD)
    if len(li_sites) >= 2:
        distances = []
        for i in range(len(li_sites)):
            for j in range(i+1, len(li_sites)):
                distances.append(li_sites[i].distance(li_sites[j]))
        results["LLSD"] = np.mean(distances)
    else:
        results["LLSD"] = 0

    return results

# Query Materials Project database with original study filters
with MPRester(API_KEY) as mpr:
    query = mpr.materials.summary.search(
        elements=["Li"],
        #num_elements=(1, 2),
        #theoretical=True,
        fields=["material_id", "structure", "nelements", "band_gap",
                "energy_above_hull", "formation_energy_per_atom"]
    )

# Process all materials and compute descriptors
data = []
for doc in tqdm(query, total=len(query)):
    try:
        struct = doc.structure
        descriptors = compute_descriptors(struct)
        descriptors.update({
            "material_id": doc.material_id,
            "formula": struct.formula,
            "num_elements": doc.nelements,             # Number of elements
            "bandgap": doc.band_gap,                   # Electronic bandgap (eV)
            "energy_above_hull": doc.energy_above_hull,  # Thermodynamic stability (eV)
            "formation_energy_per_atom": doc.formation_energy_per_atom
        })
        data.append(descriptors)
    except Exception as e:
        print(f"Error processing {doc.material_id}: {str(e)}")

# Create DataFrame and save
df = pd.DataFrame(data)
csv = df.to_csv("li_structural_descriptors.csv", index=False)

print(f"Successfully processed {len(data)}/{len(query)} materials")
print(df.head())

Retrieving SummaryDoc documents:   0%|          | 0/21756 [00:00<?, ?it/s]

  0%|          | 0/21756 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4019: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4021: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:3933: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)
100%|██████████| 21756/21756 [3:07:24<00:00,  1.93it/s]


Successfully processed 21756/21756 materials
    LLB  SBI   AFC      LASD      LLSD material_id formula  num_elements  \
0  12.0  0.0  12.0  2.037129  3.055694  mp-1018134     Li3             1   
1  12.0  0.0  12.0  2.057606  3.086410  mp-1063005     Li3             1   
2  12.0  0.0  12.0  1.545429  3.090857    mp-10173     Li2             1   
3  12.0  0.0  12.0  2.862107  3.816142   mp-976411     Li4             1   
4   6.0  0.0   6.0  2.045836  2.727781   mp-604313     Li4             1   

   bandgap  energy_above_hull  formation_energy_per_atom  
0      0.0           0.000000                   0.000000  
1      0.0           0.015671                   0.015671  
2      0.0           0.005988                   0.005988  
3      0.0           0.005889                   0.005889  
4      0.0           0.261799                   0.261799  


In [18]:
from google.colab import files
files.download('li_structural_descriptors.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Create training set

In [77]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
Li_materials_df = pd.read_csv('/content/drive/MyDrive/li_structural_descriptors_formation_energy.csv')

Mounted at /content/drive


In [78]:
len(Li_materials_df)

21756

# Pre-filtering MPD Materials

In [81]:
transition_metals = [
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",  # 3rd period
    "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",  # 4th period
    "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",        # 5th period
    "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn"        # 6th period (Superheavy elements)
]

for material_index in range(len(Li_materials_df)):
  # band gap greater than 1 eV, Ehull > 0, and contains transition metal (bool)
  if (Li_materials_df.bandgap[material_index] < 1) or \
     (not Li_materials_df.energy_above_hull[material_index] == 0) or \
     (not any(metal in Li_materials_df.formula[material_index] for metal in
         transition_metals)):
      Li_materials_df = Li_materials_df.drop(index=material_index)

len(Li_materials_df)

452

In [82]:
csv = Li_materials_df.to_csv("li_structural_descriptors_filtered.csv", index=False)
from google.colab import files
files.download('li_structural_descriptors_filtered.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Failed attempt at filtering based on Vox > 4V

In [21]:
import re

In [25]:
# formation_energy_per_atom variable in units of eV/atom in the MPD
# so Faraday constant must be represented in eV/

F_C_mol = 9.648533e4
q = 1.6e-19                 # charge of an electron
N_A = 6.022e23              # avogadro's number

In [43]:
# Find the first occurrence of the value
position = Li_materials_df.formula.isin("Li1 Cl1").stack().idxmax() if "Ba1 Li1 B1 S3" in Li_materials_df.formula.values else None
print(position)  # (row_index, column_name)

None


In [67]:
value = "Li3 In1 Cl6"

In [68]:
# Find the first occurrence of the value
position = Li_materials_df.isin([value]).stack().idxmax() if value in Li_materials_df.formula.values else None
print(position)  # (row_index, column_name)

(9981, 'formula')


In [74]:
Li_materials_df.material_id[9981]

'mp-1111288'

In [72]:
G_ox = Li_materials_df.formation_energy_per_atom.values[9981]
num_Li_atoms = int(re.search(r"Li(\d+)", Li_materials_df.formula[9981]).group(1))
V_ox = G_ox*q/(num_Li_atoms*F_C_mol/N_A)
V_ox

-0.5460622044314599